In [2]:
import datetime
import seaborn as sns
%matplotlib inline
import pandas as pd

df = pd.DataFrame({'HomeTeam': ['HOU', 'CHI', 'DAL', 'HOU'],
                   'AwayTeam' : ['CHI', 'DAL', 'CHI', 'DAL'],
                   'HomeGameNum': [1, 2, 2, 2],
                   'AwayGameNum' : [1, 1, 3, 3],
                   'Date' : [datetime.date(2014,3,11), datetime.date(2014,3,12),
                             datetime.date(2014,3,14), datetime.date(2014,3,15)]})

In [10]:
df

,AwayGameNum,AwayTeam,Date,HomeGameNum,HomeTeam
0,1,CHI,2014-03-11,1,HOU
1,1,DAL,2014-03-12,2,CHI
2,3,CHI,2014-03-14,2,DAL
3,3,DAL,2014-03-15,2,HOU


NameError: name 'df' is not defined

In [20]:
from collections import defaultdict
tree = lambda: defaultdict(tree)
d = tree()
for _, (AwayGameNum, AwayTeam, Date, HomeGameNum) in df.iterrows():
    d['AwayGameNum']['AwayTeam']['Date']['HomeGameNum']

ValueError: too many values to unpack (expected 4)

In [42]:
df['Date'] = pd.to_datetime(df.Date)

In [22]:
tidy = (df.reset_index()
           .rename(columns=dict(index='game_number'))
           .pipe(pd.melt, id_vars=['game_number', 'Date'],
                  value_vars=['AwayTeam', 'HomeTeam'])
            .sort_values(by='game_number'))

In [44]:
df = pd.read_csv('../BayesianRugby/data/results_2014.csv')
df_2 = (df.reset_index().rename(columns=dict(index='game_number'))
                .pipe(pd.melt, id_vars=['game_number', 'home_score', 'away_score'],
                     value_vars=['home_team'])
                .sort_values(by='game_number'))
df_2

,game_number,home_score,away_score,variable,value
0,0,23,15,home_team,Wales
1,1,26,24,home_team,France
2,2,28,6,home_team,Ireland
3,3,26,3,home_team,Ireland
4,4,0,20,home_team,Scotland
5,5,30,10,home_team,France
6,6,27,6,home_team,Wales
7,7,20,21,home_team,Italy
8,8,13,10,home_team,England
9,9,46,7,home_team,Ireland


In [45]:
import numpy as np
teams = df_2.value.unique()
teams = pd.DataFrame(teams, columns=['team'])
teams['i'] = teams.index
df = pd.merge(df, teams, left_on='home_team', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_home'}).drop('team', 1)
df = pd.merge(df, teams, left_on='away_team', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_away'}).drop('team', 1)


In [46]:
df

,home_team,away_team,home_score,away_score,i_home,i_away
0,Wales,Italy,23,15,0,4
1,France,England,26,24,1,5
2,Ireland,Scotland,28,6,2,3
3,Ireland,Wales,26,3,2,0
4,Scotland,England,0,20,3,5
5,France,Italy,30,10,1,4
6,Wales,France,27,6,0,1
7,Italy,Scotland,20,21,4,3
8,England,Ireland,13,10,5,2
9,Ireland,Italy,46,7,2,4


In [48]:
observed_home_goals = df.home_score.values
observed_away_goals = df.away_score.values

home_team = df.i_home.values
away_team = df.i_away.values


In [67]:
import pymc as pm
duration_of_game = 80 
avg_goals_per_team_home = observed_home_goals/(len(home_team))
avg_goals_per_team_away = observed_away_goals/(len(away_team))

avg_goals_per_team_home

array([ 1.53333333,  1.73333333,  1.86666667,  1.73333333,  0.        ,
        2.        ,  1.8       ,  1.33333333,  0.86666667,  3.06666667,
        1.13333333,  1.93333333,  0.73333333,  3.4       ,  1.33333333])

In [68]:
home_scoring_prior = pm.Exponential('home_scoring', avg_goals_per_team_home)
away_scoring_prior = pm.Exponential('away_scoring', avg_goals_per_team_away)



ZeroProbability: Stochastic home_scoring's value is outside its support,
 or it forbids its parents' current values.

In [23]:
tidy_2 = tidy.groupby('value').Date.apply(lambda x: x.diff() - pd.Timedelta(days=1)).to_frame(name='rest')

In [24]:
tidy_2

,rest
0,NaT
4,NaT
1,NaT
5,0 days
2,1 days
6,1 days
3,0 days
7,3 days


In [ ]:

!curl -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

In [ ]:

import pandas as pd
data = pd.read_csv('FremontBridge.csv', index_col='Date', parse_dates=True)
data.head()

In [ ]:
data.columns = ['West', 'East']
data.fillna(0, inplace=True)
data['Total'] = data.eval('East + West')
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
import numpy as np
data.resample('W', how='sum').plot()
plt.ylabel('weekly trips');

In [ ]:
pivoted = data.pivot_table(['East', 'West'],
                          index=data.index.date,
                          columns=data.index.hour,
                          fill_value=0)

In [ ]:
pivoted.head()
X = pivoted.values
X.shape


In [ ]:
from sklearn.decomposition import PCA
Xpca = PCA(0.9).fit_transform(X)
Xpca.shape

In [ ]:

total_trips = X.sum(1)
plt.scatter(Xpca[:, 0], Xpca[:, 1], c=total_trips,
            cmap='cubehelix')
plt.colorbar(label='total trips');

In [ ]:
from sklearn.mixture import GMM
gmm = GMM(2, covariance_type='full', random_state=0)
gmm.fit(Xpca)
cluster_label = gmm.predict(Xpca)
plt.scatter(Xpca[:, 0], Xpca[:, 1], c=cluster_label);

In [ ]:
pivoted['Cluster'] = cluster_label
data = data.join(pivoted['Cluster'], on=data.index.date)
data.head()

In [ ]:
by_hour = data.groupby(['Cluster', data.index.time]).mean()
by_hour.head()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,5))
hourly_ticks = 4 * 60 * 60 * np.arange(6)

for i in range(2):
    by_hour.ix[i].plot(ax=ax[i], xticks=hourly_ticks)
    ax[i].set_title('Cluster {0}'.format(i))
    ax[i].set_ylabel('average hourly trips')